## Cloning the repository

In [1]:
!git clone https://github.com/satwikmishra/pytorch_modules.git

fatal: destination path 'pytorch_modules' already exists and is not an empty directory.


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch 
from torchvision import datasets, transforms
from torchsummary import summary

from pytorch_modules.utils import *
from pytorch_modules.models import resnet
from pytorch_modules.main import *
from pytorch_modules.dataloader import dataloader_for_CIFAR10


from torch.utils.data import DataLoader


In [3]:
!pip install --upgrade albumentations

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

from pytorch_modules.utils.gradCAM import GradCAM

import importlib
import pytorch_modules
importlib.reload(pytorch_modules.utils.gradCAM)

<module 'pytorch_modules.utils.gradCAM' from '/content/pytorch_modules/utils/gradCAM.py'>

In [7]:
train = datasets.MNIST("",train=True,download=True,transform=train_transform)
test = datasets.MNIST("",train=False,download=True,transform=test_transform)

In [8]:
trainset = torch.utils.data.DataLoader(train,batch_size=2,shuffle=True)
testset = torch.utils.data.DataLoader(test,batch_size=2,shuffle=True)

In [9]:
cuda = torch.cuda.is_available()
print("CUDA Available:", cuda)
SEED = 1
# For reproducibility
torch.manual_seed(SEED)
BATCH_SIZE=32

CUDA Available: True


In [6]:
train_transform = A.Compose({
  A.Rotate (limit=6, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
  A.Sequential([A.CropAndPad(px=4, keep_size=False), #padding of 2, keep_size=True by default
                A.RandomCrop(32,32)]),
  A.CoarseDropout(1, 16, 16, 1, 16, 16,fill_value=0.473363, mask_fill_value=None),
  A.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784)),
})

test_transform = A.Compose({
  A.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))
})

In [18]:
trainset = datasets.MNIST("",train=True,download=True,transform=train_transform)
testset = datasets.MNIST("",train=False,download=True,transform=test_transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw



In [12]:
train_loader = DataLoader(train,batch_size=128,shuffle=True)
test_loader = DataLoader(test,batch_size=128,shuffle=True)

In [13]:
def plot_aug(aug_dict, data, ncol=6):
    nrow = len(aug_dict)

    fig, axes = plt.subplots(ncol, nrow, figsize=( 3*nrow, 15), squeeze=False)
    for i, (key, aug) in enumerate(aug_dict.items()):
        for j in range(ncol):
            ax = axes[j,i]
            if j == 0:
                ax.text(0.5, 0.5, key, horizontalalignment='center', verticalalignment='center', fontsize=15)
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)
                ax.axis('off')
            else:
                image, label = data.dataset[j-1]
                if aug is not None:
                    transform = A.Compose([aug])
                    image = np.array(image)
                    image = transform(image=image)['image']
                
                ax.imshow(image)
                #ax.set_title(f'{data.classes[label]}')
                ax.axis('off')

    plt.tight_layout()
    plt.show()

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
net_18 = resnet.ResNet18().to(device)
summary(net_18, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [16]:
optimizer = torch.optim.SGD(net_18.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,mode='min', patience=8, verbose=True)
use_l1 = False
criterion = torch.nn.CrossEntropyLoss()
net_18, history_18 = fit_model(
    net=net_18,
    optimizer=optimizer,
    criterion = criterion,
    device=device,
    no_of_epochs=40,
    train_loader = train_loader, 
    test_loader=test_loader,
    use_l1 = use_l1,
    scheduler=scheduler
)

Epoch: 1 (LR:  0.01)


  0%|          | 0/469 [00:00<?, ?it/s]


KeyError: ignored